In [1]:
import os
import glob
from datetime import datetime
from pathlib import Path
import pandas as pd

def get_latest_vigencia_file():
    base_path = Path(os.path.expanduser("~/DERCO CHILE REPUESTOS SpA/Planificación y abastecimiento - Documentos/Planificación y Compras Maestros/Vigencias"))
    pattern = str(base_path / "*Vigencia*.xlsx")
    files = glob.glob(pattern)
    
    if not files:
        return None

    def extract_date(filename):
        try:
            date_str = Path(filename).stem.split()[0]
            return datetime.strptime(date_str, "%Y-%m")
        except ValueError:
            return datetime.min

    latest_file = max(files, key=extract_date)
    return latest_file

latest_vigencia_file = get_latest_vigencia_file()

if latest_vigencia_file:
    print(f"El archivo de Vigencia más reciente es: {latest_vigencia_file}")
else:
    print("No se encontró ningún archivo de Vigencia.")


El archivo de Vigencia más reciente es: C:\Users\etorres.DERCOPARTS\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentos\Planificación y Compras Maestros\Vigencias\2024-11 Vigencia Demanda AFM.xlsx


In [2]:

def process_excel(file_path):
    df=pd.read_excel(file_path)
    columns=['Material SAP','CES 01','Mayorista','Sodimac','Easy','Walmart','SMU','Tottus','Retail (AP-AG)']
    df=df[columns]
    pivot_tables={}
    for col in columns[1:]:
        pivot=df[df[col]==1].groupby(['Material SAP',col]).size().reset_index(name='Count')
        pivot['Material SAP']=pivot['Material SAP'].astype(int)
        pivot[col]=pivot[col].astype(int)
        df_name=f"df_{col.replace(' ','_').lower()}"
        pivot_tables[df_name]=pivot
    return pivot_tables
file_path=get_latest_vigencia_file()
if file_path:
    results=process_excel(file_path)
    for df_name,table in results.items():
        print(f"Nombre del DataFrame: {df_name}")
else:
    print("No se encontró el archivo.")

Nombre del DataFrame: df_ces_01
Nombre del DataFrame: df_mayorista
Nombre del DataFrame: df_sodimac
Nombre del DataFrame: df_easy
Nombre del DataFrame: df_walmart
Nombre del DataFrame: df_smu
Nombre del DataFrame: df_tottus
Nombre del DataFrame: df_retail_(ap-ag)


In [3]:
def merge_dataframes(dataframes):
    merged = pd.DataFrame()
    for name, df in dataframes.items():
        df = df.drop(columns=['Count'], errors='ignore')
        second_col = df.columns[1]
        if 'Retail' in second_col:
            df['ID'] = df['Material SAP'].astype(str) + 'Retail'
        else:
            df['ID'] = df['Material SAP'].astype(str) + second_col
        df = df[['ID'] + df.columns[:-1].tolist()]
        merged = pd.concat([merged, df], ignore_index=True)

    # Crear la columna 'vigencia'
    merged['vigencia'] = merged.iloc[:, 2:].gt(0).any(axis=1).astype(int)
    
    # Mantener solo las columnas 'ID', 'Material SAP' y 'vigencia'
    # merged = merged[['ID', 'Material SAP', 'vigencia']]
    
    return merged

file_path = get_latest_vigencia_file()
if file_path:
    results = process_excel(file_path)
    merged_df = merge_dataframes(results)
    current_date = datetime.now().strftime("%Y-%m")
    output_path = Path(file_path).parent / f"Resumen_Vigencias_{current_date}.xlsx"
    merged_df.to_excel(output_path, index=False)
    print(f"Archivo guardado como: {output_path}")
else:
    print("No se encontró el archivo.")


Archivo guardado como: C:\Users\etorres.DERCOPARTS\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentos\Planificación y Compras Maestros\Vigencias\Resumen_Vigencias_2024-11.xlsx
